## Discover CoAP

description

### Compile the Contiki-NG firmwares

#### IoT-LAB platform support

Since IoT-LAB boards support is not included in the Contiki-NG official repository, set a `ARCH_PATH` environment variable that points to the additionnal platform support:

In [6]:
%env ARCH_PATH = /home/user/iot-lab-training/contiki-ng/iot-lab-contiki-ng/arch
%env IOTLAB_API_URL=https://devwww.iot-lab.info/rest/

env: ARCH_PATH=/home/user/iot-lab-training/contiki-ng/iot-lab-contiki-ng/arch
env: IOTLAB_API_URL=https://devwww.iot-lab.info/rest/


#### Radio settings

If you are running this training as the same time as other people, it is a good idea to change the default radio configuration to avoid too much collision with others.

Use the following cell to give you random values for channel and PAN ID.

In [2]:
import os,binascii,random
pan_id = binascii.b2a_hex(os.urandom(2)).decode()
channel = random.randint(11, 26)
print('Use CHANNEL={}, PAN_ID={}'.format(channel, pan_id))

Use CHANNEL=17, PAN_ID=c060


Change default values below before running the cell.

In [3]:
%env RADIO = IEEE802154_CONF_PANID=0xBEEF,IEEE802154_CONF_DEFAULT_CHANNEL=11

env: RADIO=IEEE802154_CONF_PANID=0xBEEF,IEEE802154_CONF_DEFAULT_CHANNEL=11


#### CoAP Server

We modified a bit the `coap-example-server`. Compile the local example.

In [4]:
!make -C coap-example-server TARGET=iotlab BOARD=m3 DEFINES=$RADIO

make: Entering directory '/home/user/iot-lab-training/contiki-ng/networking/coap/coap-example-server'
make: Leaving directory '/home/user/iot-lab-training/contiki-ng/networking/coap/coap-example-server'


#### Border Router

Use the `rpl-border-router` example, to be used by one node as border router, and enabe communication with all nodes from the frontend server.

In [5]:
%env APP_DIR = ../../iot-lab-contiki-ng/contiki-ng/examples/rpl-border-router
!make -C $APP_DIR TARGET=iotlab BOARD=m3 DEFINES=$RADIO

env: APP_DIR=../../iot-lab-contiki-ng/contiki-ng/examples/rpl-border-router
make: Entering directory '/home/user/iot-lab-training/contiki-ng/iot-lab-contiki-ng/contiki-ng/examples/rpl-border-router'
make: Leaving directory '/home/user/iot-lab-training/contiki-ng/iot-lab-contiki-ng/contiki-ng/examples/rpl-border-router'


### Launch an experiment

1. Choose your site (grenoble|lille|saclay|strasbourg|...):

In [7]:
%env SITE = devgrenoble

env: SITE=devgrenoble


2. Submit an experiment with two nodes, the compiled firware and the sniffing profile:

In [8]:
!iotlab-experiment submit -d 120 -l 3,archi=m3:at86rf231+site=$SITE

Using custom api_url: https://devwww.iot-lab.info/rest/
{
    "id": 16352
}


2. Wait for the experiment to be in the Running state:

In [9]:
!iotlab-experiment wait

Using custom api_url: https://devwww.iot-lab.info/rest/
Waiting that experiment 16352 gets in state Running
"Running"


3. Check the resources allocated to the experiment:

In [10]:
!iotlab-experiment get -ri

Using custom api_url: https://devwww.iot-lab.info/rest/
{
    "items": [
        {
            "devgrenoble": {
                "m3": "1+10+17"
            }
        }
    ]
}


And assign IDs to environment variables:

In [11]:
%env ID1 = 1
%env ID2 = 10

env: ID1=1
env: ID2=10


4. Flash the first node as border router:

In [12]:
!iotlab-node --update $APP_DIR/border-router.iotlab -l $SITE,m3,$ID1

Using custom api_url: https://devwww.iot-lab.info/rest/
{
    "0": [
        "m3-1.devgrenoble.iot-lab.info"
    ]
}


5. Flash the second node as CoAP server:

In [13]:
!iotlab-node --update coap-example-server/coap-example-server.iotlab -l $SITE,m3,$ID2

Using custom api_url: https://devwww.iot-lab.info/rest/
{
    "0": [
        "m3-10.devgrenoble.iot-lab.info"
    ]
}


### Set your private IPv6 network

In the first Terminal, launch the following command from the frontend server to create the tunslip interface like in the IPv6 training:

You will see in the output the IP address of the border router (_fd00::xxxx_).

In the second Terminal, from the frontend server again, get the routes page of the border router's web server to know the IP address of the CoAP server (_Routing links_):

### CoAP requests

You now have a connection between the frontend server and your node running the CoAP server firmware. Let's use the [aiocoap](https://aiocoap.readthedocs.io/en/latest/index.html) client from there to call server URIs.

#### Discover resources

First you need to know which ressources are avalaible. For that purpose the standard `/.well-known/core` URI is present.

You should see a list with the following resources:
- /test/hello
- /debug/mirror
- /test/chunks
- /test/separate
- /test/push
- /test/sub
- /test/b1sepb2
- /actuators/toogle
- /sensors/light

#### Get sensor value

The more standard use is to call a sensor ressource with a GET method (used by default by the client).

Another ressource is available to test the GET method. For this one you can specify a paramater thanks to a query string. Test different values, and have a look to the `coap-example-server/resources/res-hello.c` to understand what's going on.

#### Observe